<a href="https://colab.research.google.com/github/Sankarlal-MyGH/FakeNewsClassifierUsingNLP/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [7]:
df = df.dropna()

In [8]:
#Get the independent features
X=df.drop('label',axis=1)

In [9]:
X.shape

(18285, 4)

In [10]:

## Get the Dependent features
y=df['label']

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf


In [13]:
tf.__version__

'2.6.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
voc_size=5000

In [16]:
messages=X.copy()


In [17]:
messages['title'][1]


'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace=True)


In [ ]:
import nltk
import re
nltk.download('all')
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
#One Hot representation of words in corpus
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [23]:
onehot_repr[0]

[742, 4034, 862, 4074, 4858, 540, 761, 840, 1855, 3503]

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,maxlen=sent_length,padding='pre')
print(embedded_docs)

[[   0    0    0 ...  840 1855 3503]
 [   0    0    0 ...   78 3804 3724]
 [   0    0    0 ... 4895 3743  507]
 ...
 [   0    0    0 ... 4382  861 3844]
 [   0    0    0 ... 1239 1607  695]
 [   0    0    0 ... 1294 4842 1274]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  742,
       4034,  862, 4074, 4858,  540,  761,  840, 1855, 3503], dtype=int32)

In [26]:
#Building the LSTM model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
len(embedded_docs),y.shape


(18285, (18285,))

In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [31]:
X_final.shape,y_final.shape


((18285, 20), (18285,))

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [35]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 42ms/step - loss: 0.3457 - accuracy: 0.8367 - val_loss: 0.2034 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 7s 38ms/step - loss: 0.1382 - accuracy: 0.9453 - val_loss: 0.2017 - val_accuracy: 0.9130
Epoch 3/10
192/192 [==============================] - 8s 39ms/step - loss: 0.0899 - accuracy: 0.9681 - val_loss: 0.2391 - val_accuracy: 0.9148
Epoch 4/10
192/192 [==============================] - 8s 39ms/step - loss: 0.0627 - accuracy: 0.9789 - val_loss: 0.2783 - val_accuracy: 0.9123
Epoch 5/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0391 - accuracy: 0.9872 - val_loss: 0.3208 - val_accuracy: 0.9052
Epoch 6/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0209 - accuracy: 0.9938 - val_loss: 0.4381 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0170 - accuracy: 0.9946 - val_loss: 0.4871 - val_accuracy: 0.899

In [41]:
y_pred=((model.predict(X_test)> 0.5).astype("int32"))

In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)


array([[3144,  275],
       [ 309, 2307]])

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9032311516155758

In [44]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [45]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 42ms/step - loss: 0.3452 - accuracy: 0.8358 - val_loss: 0.2235 - val_accuracy: 0.9118
Epoch 2/10
192/192 [==============================] - 8s 41ms/step - loss: 0.1485 - accuracy: 0.9435 - val_loss: 0.1998 - val_accuracy: 0.9165
Epoch 3/10
192/192 [==============================] - 7s 39ms/step - loss: 0.1071 - accuracy: 0.9607 - val_loss: 0.2249 - val_accuracy: 0.9167
Epoch 4/10
192/192 [==============================] - 8s 39ms/step - loss: 0.0843 - accuracy: 0.9701 - val_loss: 0.2274 - val_accuracy: 0.9142
Epoch 5/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0640 - accuracy: 0.9781 - val_loss: 0.2675 - val_accuracy: 0.9138
Epoch 6/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0442 - accuracy: 0.9853 - val_loss: 0.3326 - val_accuracy: 0.9107
Epoch 7/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0346 - accuracy: 0.9882 - val_loss: 0.3662 - val_accuracy: 0.913

In [46]:
y_pred1=((model.predict(X_test)> 0.5).astype("int32"))

In [47]:
confusion_matrix(y_test,y_pred1)


array([[3144,  275],
       [ 309, 2307]])

In [48]:
accuracy_score(y_test,y_pred1)

0.9032311516155758